# 3.1 Manipulação e Limpeza

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('dataset/train.csv', sep=';')
data_test = pd.read_csv('dataset/test.csv', sep=';')

### Intensidade e Simetria (datasets reduzidos)

In [3]:
#label
df_reduzido = pd.DataFrame()

df_reduzido['label'] = data['label']

df_reduzido

label
0         0
1         0
2         0
3         0
4         0
...     ...
2181      5
2182      5
2183      5
2184      5
2185      5

[2186 rows x 1 columns]

intensidade é $$\frac{\sum_{x=0}^{783} pixelx}{255}$$



In [4]:
#axis = 0 é a soma das colunas; axis = 1 é a soma das linhas
def intensidade(data):
    return data.sum(axis=1) / 255

In [5]:
df_reduzido['intensidade'] = intensidade(data)

df_reduzido['intensidade']

0       145.435294
1       118.576471
2       127.600000
3       138.047059
4       146.215686
           ...    
2181    105.705882
2182    113.070588
2183    110.745098
2184     99.325490
2185    101.192157
Name: intensidade, Length: 2186, dtype: float64

simetria é

1. Simetria Vertical
$$\frac{\sum_{i=0}^{27} \sum_{j=0}^{13} || pixel_{i, j} - pixel_{i, 27-j} ||}{255}$$

2. Simetria Horizontal
$$\frac{\sum_{i=0}^{13} \sum_{j=0}^{27} || pixel_{i, j} - pixel_{27-i, j} ||}{255}$$

3. Simetria Total
$$S_{v} + S_{h}$$


In [6]:
#simetria vertical
def simetria_vertical(data):

    s_v = []
    for n in range(len(data)):
        aux = 0

        for i in range(28):

            for j in range(14):

                pixel_i = i * 28 + j
                pixel_j = i * 28 + (27 - j)
                aux += abs(data.iloc[n, pixel_i+1] - data.iloc[n, pixel_j+1])


        s_v.append(aux / 255)
    
    return s_v


#simetria horizontal
def simetria_horizontal(data):
    s_h = []
    for n in range(len(data)):
        aux = 0

        for i in range(14):

            for j in range(28):

                pixel_i = i * 28 + j
                pixel_j = (27 - i) * 28 + j
                aux += abs(data.iloc[n, pixel_i+1] - data.iloc[n, pixel_j+1])

        s_h.append(aux / 255)

    return s_h

In [7]:
#simetria total
def simetria(data):
    
    vert = simetria_vertical(data)
    hori = simetria_horizontal(data)
    total = [sv + sh for sv, sh in zip(vert, hori)]

    return total

In [8]:
df_reduzido['simetria'] = simetria(data)

In [9]:
df_reduzido

label  intensidade    simetria
0         0   145.435294  148.572549
1         0   118.576471  137.113725
2         0   127.600000  134.047059
3         0   138.047059  151.003922
4         0   146.215686  122.501961
...     ...          ...         ...
2181      5   105.705882  133.890196
2182      5   113.070588  116.698039
2183      5   110.745098  133.019608
2184      5    99.325490  132.133333
2185      5   101.192157  141.364706

[2186 rows x 3 columns]

#### Teste

In [10]:
#label
df_test_red = pd.DataFrame()

df_test_red['label'] = data_test['label']

df_test_red

label
0         0
1         0
2         0
3         0
4         0
...     ...
1110      5
1111      5
1112      5
1113      5
1114      5

[1115 rows x 1 columns]

In [11]:
#intensidade
df_test_red['intensidade'] = intensidade(data_test)

In [12]:
#simetria
df_test_red['simetria'] = simetria(data_test)

In [13]:
df_test_red

label  intensidade    simetria
0         0   147.933333  130.470588
1         0   131.054902  122.721569
2         0   134.890196  151.725490
3         0   139.976471  133.921569
4         0   129.580392  130.274510
...     ...          ...         ...
1110      5    96.141176  139.521569
1111      5    98.458824  130.729412
1112      5    95.160784  122.227451
1113      5   106.901961  128.007843
1114      5   117.658824  123.380392

[1115 rows x 3 columns]

### Novos Datasets

In [14]:
#df_reduzido.to_csv('dataset/train_redu.csv',index=False)
#df_test_red.to_csv('dataset/test_redu.csv',index=False)

# 3.2 - Classificação 1x5

In [15]:
TRAIN_PATH = 'dataset/train_redu.csv'
TEST_PATH = 'dataset/test_redu.csv'

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

### Filtragem

In [16]:
train1x5 = train[train['label'].isin([1, 5])]
test1x5 = test[test['label'].isin([1, 5])]

In [17]:
print(test.describe())
print(test1x5.describe())

             label  intensidade     simetria
count  1115.000000  1115.000000  1115.000000
mean      2.693274    93.261402   109.614758
std       2.010542    28.202407    25.462405
min       0.000000    43.929412    57.874510
25%       1.000000    69.058824    87.870588
50%       4.000000    94.031373   112.470588
75%       5.000000   107.072549   128.952941
max       5.000000   158.301961   161.035294
            label  intensidade    simetria
count  583.000000   583.000000  583.000000
mean     2.996569    77.182491  100.842498
std      2.001715    22.865517   27.015746
min      1.000000    43.929412   57.874510
25%      1.000000    54.831373   74.509804
50%      1.000000    74.568627   89.450980
75%      5.000000    97.409804  126.086275
max      5.000000   117.658824  141.992157


In [18]:
#train1x5.to_csv('dataset/train1x5.csv', index=False)
#test1x5.to_csv('dataset/test1x5.csv', index=False)

### Plot

In [19]:
train1 = train1x5[train1x5['label']==1]
train5 = train1x5[train1x5['label']==5]

fig, ax = plt.subplots()
ax.scatter(train1['intensidade'], train1['simetria'], label="1", color='b', marker='o')
ax.scatter(train5['intensidade'], train5['simetria'], label="5", color='r', marker='x')
ax.set_xlabel('intensidade')
ax.set_ylabel('simetria')
ax.legend()
plt.show()
